# 📝 HOMEWORK 2 — Pydantic (Row-Level Validation)

Bu notebook, Amazon Sales dataseti üzerinde **Pydantic v2** kullanarak satır bazlı (row-level) validation yapar.

## 🎯 Öğrenme Hedefleri
- Pydantic v2 model tanımlama
- Field validation (constraints)
- Custom validators (`@field_validator`, `@model_validator`)
- Row-level validation ile valid/invalid ayrımı
- Hata raporlama ve Slack entegrasyonu

## 🔄 Great Expectations vs Pydantic

| Özellik | Great Expectations | Pydantic |
|---------|-------------------|----------|
| Validation Seviyesi | Dataset/Column | Row/Record |
| Kullanım Alanı | Data pipelines, batches | APIs, configs, ETL |
| Çıktı | Validation report | Valid/Invalid objects |
| Hata Detayı | Aggregate stats | Per-row errors |

---
## 1️⃣ Kurulum

In [ ]:
# Kurulum (sadece ilk seferde çalıştırın)
# !pip install pydantic pandas requests

In [ ]:
# Import'lar
from pydantic import BaseModel, Field, field_validator, model_validator, ValidationError
from typing import Optional, Literal
from datetime import datetime
import pandas as pd
import requests
import json
import re

# Pydantic versiyonunu kontrol et (v2 olmalı)
import pydantic
print(f"Pydantic Version: {pydantic.__version__}")

---
## 2️⃣ Konfigürasyon

In [ ]:
# Dosya yolları
INPUT_CSV = "amazon_sales.csv"
VALID_OUTPUT_CSV = "valid_rows.csv"
INVALID_OUTPUT_CSV = "invalid_rows.csv"

# Slack Webhook URL
SLACK_WEBHOOK_URL = "YOUR_SLACK_WEBHOOK_URL"

---
## 3️⃣ Veriyi Yükle

In [ ]:
# CSV'yi yükle
df = pd.read_csv(INPUT_CSV)

print(f"📊 Dataset Boyutu: {df.shape[0]} satır, {df.shape[1]} sütun")
df.head()

---
## 4️⃣ Pydantic Model Tanımla

Her satırı validate edecek model oluşturalım.

In [ ]:
class AmazonOrder(BaseModel):
    """
    Amazon siparişi için Pydantic validation modeli.
    
    Her alan için validation kuralları tanımlanmıştır.
    """
    
    # Primary Key - zorunlu ve boş olamaz
    order_id: str = Field(..., min_length=1, description="Sipariş ID - boş olamaz")
    
    # Tarih - format kontrolü validator ile yapılacak
    date: str = Field(..., description="Sipariş tarihi (MM-DD-YY formatında)")
    
    # Status - belirli değerler içinde olmalı
    status: Literal[
        "Cancelled",
        "Shipped",
        "Shipped - Delivered to Buyer",
        "Pending",
        "Shipped - Returned to Seller",
        "Shipped - Rejected by Buyer",
        "Shipped - Returning to Seller",
        "Shipped - Out for Delivery",
        "Shipped - Picked Up"
    ] = Field(..., description="Sipariş durumu")
    
    # Fulfilment
    fulfilment: Literal["Merchant", "Amazon"] = Field(..., description="Fulfillment tipi")
    
    # Miktar - 0 veya pozitif olmalı
    qty: int = Field(..., ge=0, description="Miktar (>= 0)")
    
    # Tutar - opsiyonel (Cancelled siparişlerde boş olabilir), ama varsa >= 0
    amount: Optional[float] = Field(None, ge=0, description="Tutar (>= 0)")
    
    # Para birimi - sadece INR
    currency: Literal["INR"] = Field(..., description="Para birimi (sadece INR)")
    
    # Ülke - sadece IN
    ship_country: Literal["IN"] = Field(..., description="Gönderim ülkesi (sadece IN)")
    
    # Opsiyonel alanlar
    ship_city: Optional[str] = Field(None, description="Gönderim şehri")
    ship_state: Optional[str] = Field(None, description="Gönderim eyaleti")
    category: Optional[str] = Field(None, description="Ürün kategorisi")
    
    # =========================================================================
    # CUSTOM VALIDATORS
    # =========================================================================
    
    @field_validator('date')
    @classmethod
    def validate_date_format(cls, v: str) -> str:
        """
        Tarih formatını kontrol et (MM-DD-YY).
        """
        pattern = r'^\d{2}-\d{2}-\d{2}$'
        if not re.match(pattern, v):
            raise ValueError(f"Geçersiz tarih formatı: '{v}'. Beklenen format: MM-DD-YY")
        
        # Ek olarak geçerli bir tarih mi kontrol et
        try:
            datetime.strptime(v, '%m-%d-%y')
        except ValueError:
            raise ValueError(f"Geçersiz tarih değeri: '{v}'")
        
        return v
    
    @field_validator('order_id')
    @classmethod
    def validate_order_id_not_empty(cls, v: str) -> str:
        """
        Order ID boş string olmamalı.
        """
        if not v or v.strip() == '':
            raise ValueError("Order ID boş olamaz")
        return v.strip()
    
    @field_validator('ship_city')
    @classmethod
    def normalize_city_name(cls, v: Optional[str]) -> Optional[str]:
        """
        Şehir adını büyük harfe çevir (tutarlılık için).
        """
        if v is not None:
            return v.upper()
        return v
    
    @model_validator(mode='after')
    def validate_cancelled_order(self):
        """
        İş kuralı: Cancelled siparişlerde Qty = 0 olmalı.
        """
        if self.status == "Cancelled" and self.qty != 0:
            raise ValueError(f"Cancelled siparişlerde Qty = 0 olmalı, bulunan: {self.qty}")
        return self
    
    class Config:
        # Ekstra alanları yoksay
        extra = 'ignore'
        # Attribute'ları validate et
        validate_assignment = True

In [ ]:
# Model'i test edelim - geçerli bir kayıt
valid_order = AmazonOrder(
    order_id="405-8078784-5731545",
    date="04-30-22",
    status="Shipped",
    fulfilment="Amazon",
    qty=1,
    amount=647.62,
    currency="INR",
    ship_country="IN",
    ship_city="mumbai"  # küçük harf - validator büyük harfe çevirecek
)

print("✅ Geçerli sipariş oluşturuldu:")
print(f"   Order ID: {valid_order.order_id}")
print(f"   Ship City: {valid_order.ship_city}")  # MUMBAI olmalı

In [ ]:
# Model'i test edelim - geçersiz bir kayıt
try:
    invalid_order = AmazonOrder(
        order_id="",  # BOŞ!
        date="invalid-date",  # YANLIŞ FORMAT!
        status="Shipped",
        fulfilment="Amazon",
        qty=-1,  # NEGATİF!
        amount=100.0,
        currency="USD",  # YANLIŞ!
        ship_country="US"  # YANLIŞ!
    )
except ValidationError as e:
    print("❌ Validation hatası yakalandı:")
    print(e)

---
## 5️⃣ DataFrame Satırlarını Validate Et

Her satırı Pydantic modeli ile kontrol edelim.

In [ ]:
def validate_dataframe(df: pd.DataFrame) -> tuple[list, list, list]:
    """
    DataFrame'deki her satırı validate et.
    
    Returns:
        valid_rows: Geçerli satırlar (dict listesi)
        invalid_rows: Geçersiz satırlar (dict listesi)
        errors: Hata detayları (dict listesi)
    """
    valid_rows = []
    invalid_rows = []
    errors = []
    
    for idx, row in df.iterrows():
        try:
            # DataFrame kolonlarını model alanlarına map et
            order_data = {
                'order_id': str(row.get('Order ID', '')) if pd.notna(row.get('Order ID')) else '',
                'date': str(row.get('Date', '')) if pd.notna(row.get('Date')) else '',
                'status': row.get('Status'),
                'fulfilment': row.get('Fulfilment'),
                'qty': int(row.get('Qty', 0)) if pd.notna(row.get('Qty')) else 0,
                'amount': float(row.get('Amount')) if pd.notna(row.get('Amount')) else None,
                'currency': row.get('currency') if pd.notna(row.get('currency')) else None,
                'ship_country': row.get('ship-country') if pd.notna(row.get('ship-country')) else None,
                'ship_city': row.get('ship-city') if pd.notna(row.get('ship-city')) else None,
                'ship_state': row.get('ship-state') if pd.notna(row.get('ship-state')) else None,
                'category': row.get('Category') if pd.notna(row.get('Category')) else None,
            }
            
            # Validate et
            validated_order = AmazonOrder(**order_data)
            
            # Geçerli - orijinal satırı kaydet
            valid_rows.append(row.to_dict())
            
        except ValidationError as e:
            # Geçersiz - satırı ve hataları kaydet
            invalid_rows.append(row.to_dict())
            
            # Hata detaylarını topla
            error_details = {
                'row_index': idx,
                'order_id': row.get('Order ID', 'N/A'),
                'errors': []
            }
            
            for error in e.errors():
                error_details['errors'].append({
                    'field': '.'.join(str(loc) for loc in error['loc']),
                    'message': error['msg'],
                    'type': error['type'],
                    'input': error.get('input', 'N/A')
                })
            
            errors.append(error_details)
    
    return valid_rows, invalid_rows, errors

In [ ]:
# Validation çalıştır
print("🔄 Validation çalıştırılıyor...\n")

valid_rows, invalid_rows, errors = validate_dataframe(df)

print("=" * 60)
print("   VALIDATION SONUCU")
print("=" * 60)
print(f"\n   📊 Toplam Satır: {len(df)}")
print(f"   ✅ Geçerli: {len(valid_rows)}")
print(f"   ❌ Geçersiz: {len(invalid_rows)}")
print(f"   📈 Başarı Oranı: {len(valid_rows)/len(df)*100:.1f}%")
print("=" * 60)

---
## 6️⃣ Hata Detaylarını İncele

In [ ]:
# Hataları detaylı göster
if errors:
    print("\n" + "=" * 60)
    print("   ❌ HATA DETAYLARI")
    print("=" * 60)
    
    for err in errors:
        print(f"\n📌 Satır {err['row_index']} (Order ID: {err['order_id']})")
        print("-" * 40)
        
        for e in err['errors']:
            print(f"   ⚠️  Alan: {e['field']}")
            print(f"      Hata: {e['message']}")
            print(f"      Değer: {e['input']}")
            print()
else:
    print("\n🎉 Tüm satırlar geçerli!")

In [ ]:
# Hata istatistikleri
if errors:
    print("\n📊 HATA İSTATİSTİKLERİ")
    print("-" * 40)
    
    # Hangi alanlarda hata var?
    field_errors = {}
    for err in errors:
        for e in err['errors']:
            field = e['field']
            if field not in field_errors:
                field_errors[field] = 0
            field_errors[field] += 1
    
    # Sırala ve göster
    for field, count in sorted(field_errors.items(), key=lambda x: -x[1]):
        print(f"   {field}: {count} hata")

---
## 7️⃣ Sonuçları CSV'ye Kaydet

In [ ]:
# Geçerli satırları kaydet
if valid_rows:
    valid_df = pd.DataFrame(valid_rows)
    valid_df.to_csv(VALID_OUTPUT_CSV, index=False)
    print(f"✅ Geçerli satırlar kaydedildi: {VALID_OUTPUT_CSV} ({len(valid_rows)} satır)")
else:
    print("⚠️  Geçerli satır yok!")

In [ ]:
# Geçersiz satırları kaydet (hata nedenleri ile)
if invalid_rows:
    invalid_df = pd.DataFrame(invalid_rows)
    
    # Hata nedenlerini ekle
    error_reasons = []
    for err in errors:
        reasons = " | ".join([f"{e['field']}: {e['message']}" for e in err['errors']])
        error_reasons.append(reasons)
    
    invalid_df['validation_errors'] = error_reasons
    invalid_df.to_csv(INVALID_OUTPUT_CSV, index=False)
    print(f"❌ Geçersiz satırlar kaydedildi: {INVALID_OUTPUT_CSV} ({len(invalid_rows)} satır)")
else:
    print("🎉 Geçersiz satır yok!")

In [ ]:
# Geçersiz satırları incele
if invalid_rows:
    print("\n📋 Geçersiz Satırlar (validation_errors kolonu ile):")
    invalid_df = pd.read_csv(INVALID_OUTPUT_CSV)
    display(invalid_df[['Order ID', 'Date', 'Status', 'Qty', 'Amount', 'currency', 'ship-country', 'validation_errors']])

---
## 8️⃣ Slack Bildirimi Gönder

In [ ]:
def send_slack_notification(webhook_url: str, total: int, valid: int, invalid: int, error_summary: dict) -> bool:
    """
    Pydantic validation sonuçlarını Slack'e gönder.
    """
    if webhook_url == "YOUR_SLACK_WEBHOOK_URL":
        print("⚠️  Slack bildirimi atlandı (webhook URL yapılandırılmamış)")
        return False
    
    # Durum belirleme
    if invalid == 0:
        emoji = "✅"
        color = "#36a64f"
        status_text = "ALL VALID"
    else:
        emoji = "⚠️"
        color = "#ffc107"
        status_text = f"{invalid} INVALID ROWS"
    
    # Hata özeti
    error_text = ""
    if error_summary:
        error_text = "\n".join([f"• `{field}`: {count} hata" for field, count in error_summary.items()])
    
    # Mesaj oluştur
    message = {
        "attachments": [
            {
                "color": color,
                "blocks": [
                    {
                        "type": "header",
                        "text": {
                            "type": "plain_text",
                            "text": f"{emoji} Pydantic Validation: {status_text}",
                            "emoji": True
                        }
                    },
                    {
                        "type": "section",
                        "fields": [
                            {"type": "mrkdwn", "text": f"*Dataset:*\nAmazon Sales"},
                            {"type": "mrkdwn", "text": f"*Zaman:*\n{datetime.now().strftime('%Y-%m-%d %H:%M')}"},
                            {"type": "mrkdwn", "text": f"*Toplam:*\n{total} satır"},
                            {"type": "mrkdwn", "text": f"*Geçerli:*\n{valid} ✓"},
                            {"type": "mrkdwn", "text": f"*Geçersiz:*\n{invalid} ✗"},
                            {"type": "mrkdwn", "text": f"*Başarı:*\n{valid/total*100:.1f}%"}
                        ]
                    }
                ]
            }
        ]
    }
    
    # Hata özeti ekle
    if error_text:
        message["attachments"][0]["blocks"].append({
            "type": "section",
            "text": {
                "type": "mrkdwn",
                "text": f"*Hata Dağılımı:*\n{error_text}"
            }
        })
    
    # Gönder
    try:
        response = requests.post(
            webhook_url,
            data=json.dumps(message),
            headers={"Content-Type": "application/json"}
        )
        
        if response.status_code == 200:
            print("✅ Slack bildirimi başarıyla gönderildi!")
            return True
        else:
            print(f"❌ Slack bildirimi başarısız: {response.status_code}")
            return False
            
    except Exception as e:
        print(f"❌ Slack bildirimi hatası: {e}")
        return False

In [ ]:
# Hata özetini hazırla
field_errors = {}
for err in errors:
    for e in err['errors']:
        field = e['field']
        if field not in field_errors:
            field_errors[field] = 0
        field_errors[field] += 1

# Slack bildirimi gönder
send_slack_notification(
    webhook_url=SLACK_WEBHOOK_URL,
    total=len(df),
    valid=len(valid_rows),
    invalid=len(invalid_rows),
    error_summary=field_errors
)

---
## 9️⃣ Final Özet

In [ ]:
print("\n" + "=" * 60)
print("   📋 PYDANTIC VALIDATION - FİNAL RAPOR")
print("=" * 60)
print(f"\n   📥 Input:  {INPUT_CSV} ({len(df)} satır)")
print(f"   ✅ Output: {VALID_OUTPUT_CSV} ({len(valid_rows)} satır)")
print(f"   ❌ Output: {INVALID_OUTPUT_CSV} ({len(invalid_rows)} satır)")
print(f"\n   📈 Başarı Oranı: {len(valid_rows)/len(df)*100:.1f}%")

if field_errors:
    print(f"\n   🔍 En Çok Hata Alan Alanlar:")
    for field, count in sorted(field_errors.items(), key=lambda x: -x[1])[:3]:
        print(f"      • {field}: {count}")

print("\n" + "=" * 60)

---
## 📚 Özet

Bu notebook'ta öğrendiklerimiz:

1. **Pydantic v2 BaseModel** tanımlama
2. **Field constraints**: `min_length`, `ge`, `Literal`
3. **Custom validators**: `@field_validator`, `@model_validator`
4. **ValidationError** handling
5. **Row-level validation** ile valid/invalid ayrımı
6. **Hata raporlama** ve CSV'ye kaydetme

### 🔄 GE vs Pydantic - Ne Zaman Hangisi?

| Senaryo | Araç |
|---------|------|
| Büyük veri batch validation | Great Expectations |
| API request validation | Pydantic |
| Config dosyası validation | Pydantic |
| Statistical data checks | Great Expectations |
| Row-by-row ETL validation | Pydantic |
| Data Docs / documentation | Great Expectations |

### 🏋️ Bonus Alıştırmalar

1. `ship-postal-code` için 6 haneli format validator ekleyin
2. `email` alanı ekleyin ve email format validator yazın
3. `@model_validator` ile "Shipped" status için Amount > 0 kuralı ekleyin

---
## 🔗 Kaynaklar

- [Pydantic v2 Documentation](https://docs.pydantic.dev/latest/)
- [Pydantic Validators](https://docs.pydantic.dev/latest/concepts/validators/)
- [Field Types](https://docs.pydantic.dev/latest/concepts/fields/)
- [Error Handling](https://docs.pydantic.dev/latest/concepts/errors/)